# 6장 - 메타클래스와 애트리뷰트


- 파이썬의 특성을 열거할 때 메타클래스를 자주 언급하는데, 실제로 메타클래스가 어떤 목적으로 쓰이는지 이해하는 프로그래머는 거의 없다.
- <font color='blue'>메타클래스</font> 라는 이름은 어렴풋이 이 개념이 클래스를 넘어서는 것임을 암시한다.
- 간단히 말해, 메타클래스를 사용하면 파이썬의 class 문을 가로채서 클래스가 정의될 때마다 특별한 동작을 제공할 수 있다.
<br>
<br>
- 메타클래스처럼 신비롭고 강력한 파이썬 기능으로는 "동적으로 애트리뷰트 접근을 커스텀화해주는 내장 기능" 을 들 수 있다.
- 파이썬의 객체지향적인 요소와 방금 말한 두 기능이 함께 어우러지면 간단한 클래스를 복잡한 클래스로 쉽게 변환할 수 있다.
<br>
<br>
- 하지만, 이런 강력함에는 많은 함정이 뒤따른다.
- 동적인 애트리뷰트로 객체를 오버라이드하면 예기치 못한 부작용이 생길 수 있다.
- <font color='blue'>최소 놀람의 법칙(rule of least surprise)을 따르고 잘 정해진 관용어로만 이런 기능을 사용하는 것이 중요하다.</font>

### BETTER WAY 44 - 세터와 게터 메서드 대신 평범한 애트리뷰트를 사용하라

- 다른 언어를 사용하다 파이썬을 접한 프로그래머들은 클래스에 getter 나 setter 를 명시적으로 정의하곤 한다.

In [1]:
class OldResistor:
    def __init__(self, ohms):
        self._ohms = ohms
        
    def get_ohms(self):
        return self._ohms
    
    def set_ohms(self, ohms):
        self._ohms = ohms
        
r0 = OldResistor(50e3)
print(r0.get_ohms())
r0.set_ohms(10e3)
print(r0.get_ohms())

50000.0
10000.0


- 세터와 게터는 사용하기는 쉽지만, 이런 코드는 파이썬답지 않다.
- 대신 다음 코드와 같이 항상 단순한 공개 애트리뷰트로부터 구현을 시작하라

In [2]:
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        
r1 = Resistor(50e3)
r1.ohms = 10e3

- 나중에 애트리뷰트가 설정될 때 특별한 기능을 수행해야 한다면, 애트리뷰트를 @property 데코레이터와 대응하는 setter 애트리뷰트로 옮겨갈 수 있다.

In [3]:
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
        
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms
        
r2 = VoltageResistance(1e3)
print(f'이전: {r2.current:.2f} 암페어')
r2.voltage = 10
print(f'이후: {r2.current:.2f} 암페어')

이전: 0.00 암페어
이후: 0.01 암페어


- 프로퍼티에 setter 를 지정하면 타입을 검사하거나 클래스 프로퍼티에 전달된 값에 대한 검증을 수행할 수 있다.

In [4]:
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError(f'저항은 0보다 큰 값이어야 합니다. 실제 값: {ohms}')
        self._ohms = ohms
        
r3 = BoundedResistance(1e3)
r3.ohms = 0

ValueError: 저항은 0보다 큰 값이어야 합니다. 실제 값: 0

### BETTER WAY 45 - 애트리뷰트를 리팩터링하는 대신 @property 를 사용하라

- 내장 @property 데코레이터를 사용하면, 겉으로는 단순한 애트리뷰트처럼 보이지만 실제로는 지능적인 로직을 수행하는 애트리뷰트를 정의할 수 있다.
<br>
<br>
- @property 의 고급 활용법이자 흔히 사용하는 기법으로는, 간단한 수치 애트리뷰트를 그때그때 요청에 따라 계산해 제공하도록 바꾸는 것을 들 수 있다.
- 이 기법은 기존 클래스를 호출하는 코드를 전혀 바꾸지 않고도 클래스 애트리뷰트의 기존 동작을 변경할 수 있기 때문에 아주 유용하다.
- 특히 이 방법은 이 클래스를 호출하는 코드 중에 여러분이 제어할 수 없는 코드가 더 많은 경우 더욱 유용하다.
<br>
<br>
- @property 는 인터페이스를 점차 개선해나가는 과정에서 중간중간 필요한 기능을 제공하는 수단으로로 유용하다.

In [6]:
# 예) 일반 파이썬 객체를 사용해 리키 버킷(leaky bucket) 흐름 제어 알고리즘을 구현한다고 하자
# 다음 코드의 Bucket 클래스는 남은 가용 용량(quota)과 이 가용 용량의 잔존 시간을 표현

from datetime import datetime, timedelta

class Bucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.quota = 0
        
    def __repr__(self):
        return f'Bucket(quota={self.quota})'
    
# 리키 버킷 알고리즘은 시간을 일정한 간격(주기)으로 구분하고, 
# 가용 용량을 소비할 때마다 시간을 검사해서 주기가 달라질 경우에는 이전 주기에 미사용한 가용 용량이 새로운 주기로 넘어오지 못하게 막는다.
def fill(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = 0
    bucket.quota += amount
        
# 어떤 작업을 하고 싶을 땐 먼저 리키 버킷으로부터 자신의 작업에 필요한 용량을 할당받아야 한다.
def deduct(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        return False # 새 주기가 시작됐는데 아직 버킷 할당량이 재설정되지 않았다.
    if bucket.quota - amount < 0:
        return False # 버킷의 가용 용량이 충분하지 못하다.
    else:
        bucket.quota -= amount
        return True # 버킷의 가용 용량이 충분하므로 필요한 분량을 사용한다.
        
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

Bucket(quota=100)


In [7]:
deduct(bucket, 60)
print(bucket)
deduct(bucket, 60)
print(bucket)

Bucket(quota=40)
Bucket(quota=40)


In [11]:
# 이 구현의 문제점은 버킷이 시작할 때 가용 용량이 얼마인지 알 수 없다는 것이다.
# 이를 해결하기 위해 이번 주기에 재설정된 가용 용량인 max_quota 와 이번 주기에 버킷에서 소비한 용량의 합계인 quota_consumed 를 추적하도록 클래스를 변경할 수 있다.

class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
        
    def __repr__(self):
        return (f'NewBucket(max_quota={self.max_quota}, quota_consumed={self.quota_consumed})')
    
    # 원래의 Bucket 클래스와 인터페이스를 동일하게 제공하기 위해 @property 데코레이터가 붙은 메서드를 사용해
    # 클래스의 두 애트리뷰트(max_quota와 quota_consumed) 에서 현재 가용 용량 수준을 그때그때 계산하게 한다.
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    # fill 과 deduct 함수가 quota 애트리뷰트에 값을 할당할 때는 NewBucket 클래스의 현재 사용 방식에 맞춰 특별한 동작을 수행해야 한다.
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # 새로운 주기가 되고 가용 용량을 재설정하는 경우
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # 새로운 주기가 되고, 가용 용량을 추가하는 경우
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            # 어떤 주기 안에서 가용 용량을 소비하는 경우
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta
            
bucket = NewBucket(60)
print('최초', bucket)
fill(bucket, 100)
print('보충 후', bucket)

최초 NewBucket(max_quota=0, quota_consumed=0)
보충 후 NewBucket(max_quota=100, quota_consumed=0)


In [12]:
if deduct(bucket, 99):
    print('99 용량 사용')
else:
    print('가용 용량이 작아서 99 용량을 처리할 수 없음')
print('사용 후', bucket)

if deduct(bucket, 10):
    print('10 용량 사용')
else:
    print('가용 용량이 작아서 10 용량을 처리할 수 없음')
print('여전히', bucket)

99 용량 사용
사용 후 NewBucket(max_quota=100, quota_consumed=99)
가용 용량이 작아서 10 용량을 처리할 수 없음
여전히 NewBucket(max_quota=100, quota_consumed=99)


- @property 를 사용하면 데이터 모델을 점진적으로 개선할 수 있으므로 필자는 특히 @property를 좋아한다.

### BETTER WAY 46 - 재사용 가능한 @property 메서드를 만들려면 디스크립터를 사용하라